# ゼロから作る Deep Learning 4 強化学習編 勉強ノート 第5章 〜モンテカルロ法〜

環境のモデルが既知の場合は DP で最適価値関数と最適方策を得ることができた。  
しかし、環境のモデルが未知の場合があるため、すべての問題で扱えるわけではない。  
ここでは、データのサンプリングを繰り返し行って、その結果から推定する「<font color="red">**モンテカルロ法**</font>」について扱う。

## モンテカルロ法の基礎

強化学習では、モンテカルロ法(Monte Carlo Method)を使うことで、経験から価値関数を推定することができる。  
ここに出てくる「経験」とは、具体的には環境とエージェントがやり取りを行って得られた「状態、行動、報酬」の一連のデータのことを指す。

さて、今まで扱ってきたグリッドワールドの問題などとは異なり、現実にある多くの問題では環境のモデルを知ることはできない。  
例えば、「商品の在庫管理」を行う問題を考えたときに、「商品がどれだけ売れるか」ということが環境の状態遷移確率に相当する。  
しかし、商品の売れ行きは様々な要因が複雑に絡み合っているため、それを完全に知ることは現実的には不可能である。  
もし、環境の状態遷移確率を理論的に知ることができたとしても、そのために必要な計算は多くの場合大変である。

### サイコロの目の和(*コーディング*)

ここでは、2つのサイコロのを振る問題を考える。  
サイコロの各目の出る確率は均一に $\frac{1}{6}$ とする。  
すると、確率分布表は次のようになる。

| サイコロの<br>目の和 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| 確率 | $\frac{1}{36}$ | $\frac{2}{36}$ | $\frac{3}{36}$ | $\frac{4}{36}$ | $\frac{5}{36}$ | $\frac{6}{36}$ | $\frac{5}{36}$ | $\frac{4}{36}$ | $\frac{3}{36}$ | $\frac{2}{36}$ | $\frac{1}{36}$ |

これの期待値を計算してみる。

In [ ]:
# サイコロの確率分布
ps = {
    2: 1/36, 3: 2/36, 4: 3/36, 5: 4/36, 6: 5/36, 7: 6/36,
    8: 5/36, 9: 4/36, 10: 3/36, 11: 2/36, 12: 1/36
}

# 期待値の計算
V = 0
for x, p in ps.items():
    V += x * p
print(V)

このように、確率分布が分かれば期待値の計算を行うことができる。

### 分布モデルとサンプルモデル(*コーディング*)

サイコロを振るという試行を「確率分布」としてモデル化した。  
このように確率分布として表されたモデルは<font color="red">**分布モデル**</font>(Distribution Model)と呼ぶことができる。  
一方で、サンプリングさえできれば良いという<font color="red">**サンプルモデル**</font>(Sample Model)というものがある。  
サイコロの例でいうと、実際にサイコロを振り、その目の和を観測するという方法である。  
分布モデルが確率分布を明示的に保持できることが条件であるのに対し、サンプルモデルはサンプリングできることが条件である。  
それではサンプルモデルを実装し、2つのサイコロの目の和を観測する。

In [ ]:
# ライブラリのインポート
import numpy as np

In [ ]:
# サイコロの目の和を出す関数を定義
def sample(dices: int=2) -> int:
    x = 0
    for _ in range(dices):
        x += np.random.choice([1, 2, 3, 4, 5, 6])
    return x

In [ ]:
# 実験をする
sample_a = sample()
print(f"sample a: {sample_a}")
sample_b = sample()
print(f"sample b: {sample_b}")
sample_c = sample()
print(f"sample c: {sample_c}")

このように2つのサイコロを「サンプルモデル」として実装することができた。  
これを使って期待値を計算してみる。

### モンテカルロ法の実装(*コーディング*)

モンテカルロ法によって期待値を求めるコードを書いてみる。

In [ ]:
# サンプリング回数
trial = 1000

# サンプルを繰り返し観測する
samples = []
for _ in range(trial):
    s = sample()
    samples.append(s)

# 期待値を算出する
V = sum(samples) / len(samples)
print(V)

7が期待値として正解なので、おおよそ正しいことがわかる。  
モンテカルロ法は、サンプル数を増やすことで信頼性が高まる。  
すなわち、分散が小さくなるということだ。  
続いて、サンプルデータを得るたびに平均値を求めたい場合を考える。

In [ ]:
# ライブラリのインポート
import matplotlib.pyplot as plt
from numpy.typing import ArrayLike

In [ ]:
# サンプリングした結果をプロットする関数を定義
def plot_sampling(x: ArrayLike, y: ArrayLike) -> None:
    plt.figure(figsize=(18, 12))

    plt.xlabel("samples")
    plt.xlim(min(x) - 1, max(x) + 1)
    plt.grid()

    plt.plot(x, y)
    plt.show()

In [ ]:
# サンプリング回数
trial = 1000

# サンプルを繰り返し観測し、毎回期待値を算出する
samples = []
means = []
for _ in range(trial):
    s = sample()
    samples.append(s)
    V = sum(samples) / len(samples)
    means.append(V)

# 繰り返しごとの期待値の推移をプロット
x_axis = [i for i in range(1, 1001)]
plot_sampling(x_axis, means)

これをみると、だんだん期待値が7に収束していることがわかる。  
次は、強化学習の問題に対してモンテカルロ法を適用する。

## モンテカルロ法による方策評価

強化学習の問題にモンテカルロ法を適用するには、エージェントが実際に行動して得た経験から価値関数を推定するということを考える。  
ここでは、方策 $\pi$ が与えられたときに、その方策の価値関数をモンテカルロ法を使って計算する。

### 価値関数をモンテカルロ法で求める

価値関数は次の式で表された。

$$v_{\pi}(s) = \mathbb{E}_\pi[G|s]$$

ここでは、状態 $s$ からスタートして得られる収益を $G$ で表す。  
価値関数 $c_\pi(s)$ は方策 $\pi$ に従って行動したときに得られる収益の期待値として定義される。  
ここでは、エピソードタスクを想定し、ある時刻にはゴールに辿り着くことを考える。  
モンテカルロ法を使って計算するには、方策 $\pi$ に従ってエージェントに行動させ、得られた実際の収益をサンプルデータとする。

$$V_\pi(s) = \frac{G^{(1)} + G^{(2)} + \cdots + G^{(n)}}{n}$$

$i$ 回目のエピソードで得られた収益を $G^{(i)}$ と表記している。  
モンテカルロ法で計算するには、 $n$ 回のエピソードを行い、そこで得られたサンプルデータの平均を求める。

### すべての状態の価値関数を求める

ここでは、モンテカルロ法を使ってすべての状態において価値関数を求めてみる。  
単純に考えれば、開始する状態を変更して、1つの状態だけの価値関数を求めることを繰り返せば達成できるが、それでは計算効率上の問題がある。  
それは、ある1つの状態からスタートして得られたサンプルデータは、その価値関数を計算するためだけに使われており、他の状態の価値関数の計算には貢献していないということだ。  
例えば、3つの状態 $A, B, C$ があり、 $A \rightarrow B \rightarrow C$ の順に状態を経てゴールに辿り着くとする。  
その間に得られる報酬を $R_A, R_B, R_C$ とし、このタスクでの割引率を $\gamma$ とすると、状態 $A$ からスタートして得られる収益は次のように表せる。

$$G_A = R_A + \gamma R_B + \gamma^2 R_C$$

続いて状態 $B$ から下の遷移を考えると、これは状態 $B$ からスタートした場合の収益サンプルデータと見なせるので、この収益は次のように表せる。

$$G_B = R_B + \gamma R_C$$

同様に考えると、状態 $C$ からスタートして得られる収益は

$$G_C = R_C$$

と考えられる。  
このように、1つの施行だけから3つの状態に対する収益が得られる。

### モンテカルロ法の効率の良い実装

先ほどの例から、次の3つの収益を計算する必要がある。

\begin{eqnarray*}
G_A &=& R_A + \gamma R_B + \gamma^2 R_C \\
G_B &=& R_B + \gamma R_C \\
G_C &=& R_C
\end{eqnarray*}

ここで特に難しい計算はないが、 $G_C \rightarrow G_B \rightarrow G_A$ の順番で次のように式変形することができる。

\begin{eqnarray*}
G_C &=& R_C \\
G_B &=& R_B + \gamma G_C \\
G_A &=& R_A + \gamma G_B
\end{eqnarray*}

このように後ろから順に計算することで効率化できる。

## モンテカルロ法の実装

前章で解いた「3×4のグリッドワールド」の問題をモンテカルロ法で解いてみる。  
`GridWorld` クラスに新しく `step` というメソッドを加える。

### step メソッド(*コーディング*)

今回は、環境のモデルを使わずに方策評価を行うので、エージェントに実際に行動を行わせるための `step` メソッドが必要になる。

In [ ]:
# step メソッドと reset メソッドを追加した GridWorld クラスの実装
class GridWorld:
    def __init__(self):
        self.action_space = [0, 1, 2, 3]
        self.action_meaning = {
            0: "UP",
            1: "DOWN",
            2: "LEFT",
            3: "RIGHT"
        }
        self.reward_map = np.array(
            [
                [0, 0, 0, 1.0],
                [0, None, 0, -1.0],
                [0, 0, 0, 0]
            ]
        )
        self.goal_state = (0, 3)
        self.wall_state = (1, 1)
        self.start_state = (2, 0)
        self.agent_state = self.start_state

    @property
    def height(self):
        return len(self.reward_map)

    @property
    def width(self):
        return len(self.reward_map[0])

    @property
    def shape(self):
        return self.reward_map.shape

    def actions(self):
        return self.action_space

    def states(self):
        for h in range(self.height):
            for w in range(self.width):
                yield (h, w)

    def next_state(self, state, action):
        action_move_map = [
            (-1, 0), (1, 0), (0, -1), (0, 1)
        ]
        move = action_move_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        ny, nx = next_state

        if nx < 0 or nx >= self.width or ny < 0 or ny >= self.height:
            next_state = state
        elif next_state == self.wall_state:
            next_state = state

        return next_state

    def reward(self, state, action, next_state):
        return self.reward_map[next_state]

    def step(self, action):
        state = self.agent_state
        next_state = self.next_state(state, action)
        reward = self.reward(state, action, next_state)
        done = (next_state == self.goal_state)

        self.agent_state = next_state

        return next_state, reward, done

    def reset(self):
        self.agent_state = self.start_state
        return self.agent_state

In [ ]:
# パラメータの設定
env = GridWorld()
action = 0

# step メソッドを使う
next_state, reward, done = env.step(action)
print(f"next_state:\t{next_state}")
print(f"reward:\t{reward}")
print(f"done:\t{done}")

この `step` メソッドを使ってエージェントに行動を行わせ、サンプルデータを得る。  
また、 `reset` メソッドを使うことで、最初の状態にリセットする。

In [ ]:
# GridWorld クラスのインスタンスを生成
env = GridWorld()

# 状態をリセット
state = env.reset()

### エージェントクラスの実装(*コーディング*)

続いてモンテカルロ法を使って方策評価を行うエージェントを実装する。  
このエージェントは、ランダムな方策に従って行動するものとする。  
エージェントを `RandomAgent` クラスとして実装する。

In [ ]:
# ライブラリのインポート
from collections import defaultdict

In [ ]:
# RandomAgent クラスの実装
class RandomAgent:
    def __init__(self):
        self.gamma = 0.9
        self.action_size = 4

        random_actions = {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
        self.pi = defaultdict(lambda: random_actions)
        self.V = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = []

    def get_actions(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def add(self, state, action, reward):
        data = (state, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    def eval(self):
        G = 0
        for data in reversed(self.memory):
            state, action, reward = data
            G = self.gamma * G + reward
            self.cnts[state] += 1
            self.V[state] += (G - self.V[state]) / self.cnts[state]

### モンテカルロ法を動かす(*コーディング*)

準備が整ったので、エージェントの `RandomAgent` クラスと環境の `GridWorld` クラスを連携させて動かしてみる。  
ここでは、1000回のエピソードを実行している。  
エピソードの冒頭で環境とエージェントのリセットを行い、 while ループの中でエージェントに行動させる。  
その過程で得られた「状態、行動、報酬」のサンプルデータを記録する。  
ゴールに辿り着いたら得られたサンプルデータをもとに、モンテカルロ法で価値関数を更新する。  
最後に while ループを抜けて次のエピソードを始める。

In [ ]:
# 各クラスのインスタンスを生成
env = GridWorld()
agent = RandomAgent()

# 1000回のエピソードで実行
episodes = 1000
for episode in range(episodes):
    state = env.reset()
    agent.reset()

    while True:
        action = agent.get_actions(state)
        next_state, reward, done = env.step(action)

        agent.add(state, action, reward)
        if done:
            agent.eval()
            break

        state = next_state

print(agent.V)

モンテカルロ法を使った場合も DP のときと似たような結果が得られた。

## モンテカルロ法による方策制御

方策評価をしたら、次は最適方策を見つける「方策制御」をモンテカルロ法で行うことを考える。  
前章の方策反復法で鍵となるアイデアは学んでいるので、特段新しいことは多くない。

### 評価と改善

最適方策は「評価」と「改善」を交互に繰り返すことによって得られる。  
「評価」のフェーズでは、方策を評価して価値関数を得る。  
「改善」のフェーズでは、価値関数を greedy 化することで方策を改善する。  
この2つのフェーズを交互に繰り返すことで最適方策へと近づいていく。


前章でモンテカルロ法を使って方策の評価を行った。  
例えば、 $\pi$ という方策があれば $V_\pi(s)$ を得ることができた。  
改善のフェーズでは次のような greedy 化を行う。

\begin{eqnarray*}
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\mu(s) &=& \argmax_a Q(s, a) \\
&=& \argmax_a \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma V(s') \right\}
\end{eqnarray*}

この式の問題点として、一般的な強化学習の問題では環境のモデルを知ることが出来ないということが挙げられる。  
式の内部に $p(s'|s, a)$ と $r(s, a, s')$ があるため計算が出来ない。  
そこで、式の1行目に戻って Q 関数 $Q(s, a)$ のみを使って方策の改善を行う必要がある。  
これは、単に $Q(s, a)$ が最大となる $a$ を取り出すだけなので環境のモデルを必要としない。  
Q 関数を対象に改善をする場合、モンテカルロ法の更新式を $V(s)$ から $Q(s, a)$ に切り替える。

---

**【状態価値関数に関しての評価】**

\begin{eqnarray*}
\text{一般的な方式} &:& V_n(s) = \frac{G^{(1)} + G^{(2)} + \cdots + G^{(n)}}{n} \\
\text{インクリメンタルな方式} &:& V_n(s) = V_{n-1}(s) + \frac{1}{n} \left\{ G^{(n)} - V_{n-1}(s) \right\}
\end{eqnarray*}

---

**【Q 関数に関しての評価】**

\begin{eqnarray*}
\text{一般的な方式} &:& Q_n(s, a) = \frac{G^{(1)} + G^{(2)} + \cdots + G^{(n)}}{n} \\
\text{インクリメンタルな方式} &:& Q_n(s, a) = Q_{n-1}(s, a) + \frac{1}{n} \left\{ G^{(n)} - Q_{n-1}(s, a) \right\}
\end{eqnarray*}

---

上式の通り、状態価値関数であっても Q 関数であってもモンテカルロ法で行う計算自体は変わらない。

### モンテカルロ法を使った方策制御の実装(*コーディング*)

ここでは、モンテカルロ法を使って方策制御を行うエージェントの `McAgent` クラスを実装する。

In [ ]:
# 行動の確率分布を返す関数を定義
def greedy_probs(
        Q: dict, state: tuple[int, int], action_size: int=4
    ) -> dict:
    qs = [Q[(state, action)] for action in range(action_size)]
    max_action = np.argmax(qs)

    action_probs = {action: 0.0 for action in range(action_size)}
    action_probs[max_action] = 1

    return action_probs

In [ ]:
# McAgent クラスの実装
class McAgent:
    def __init__(self):
        self.gamma = 0.9
        self.action_size = 4

        random_actions = {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
        self.pi = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = []

    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def add(self, state, action, reward):
        data = (state, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    def update(self):
        G = 0
        for data in reversed(self.memory):
            state, action, reward = data
            G = self.gamma * G + reward
            key = (state, action)
            self.cnts[key] += 1
            self.Q[key] += (G - self.Q[key]) / self.cnts[key]
            self.pi[state] = greedy_probs(self.Q, state)

`greedy_probs()` 関数が返すのは、 greedy な行動を取る確率分布、つまり、状態 `state` において Q 関数の値が最大の行動だけを取る確率分布である。  
この関数は、今後他のクラスからも利用するため `McAgent` クラスの外に定義している。

実はこの実装ではあまり上手くいかない。  
改善点は以下の2点である。

- `greedy_probs()` 関数の最後を、完全な greedy ではなく ε-greedy にする
- `McAgent` クラスの `update()` メソッドの Q の更新は「固定値 $\alpha$ 方式」で行う

### ε-greedy 法(*コーディング*)

greedy な行動を行った場合、エージェントの辿るルートはただ1つに固定されてしまう。  
これでは全ての状態と行動の組み合わせに対して収益のサンプルデータを集めることが出来ない。  
そこで ε-greedy 法を使って、エージェントの行動にランダム性を少しだけ加えることで、基本的には Q 関数が最大の行動を選び、低い確率でランダムな行動を選ぶようにする。

In [ ]:
# ε-greedy 法に直した greedy_probs 関数を定義
def greedy_probs(
        Q: dict, state: tuple[int, int], eps: float=0, action_size: int=4
    ) -> dict:
    qs = [Q[(state, action)] for action in range(action_size)]
    max_action = np.argmax(qs)

    base_prob = eps / action_size
    action_probs = {action: base_prob for action in range(action_size)}
    action_probs[max_action] += (1 - eps)

    return action_probs

### 固定値 $\alpha$ 方式へ(*コーディング*)

`McAgent` クラスの `update()` メソッドを修正する。

In [ ]:
# update() メソッドが修正された McAgent クラスの実装
class McAgent:
    def __init__(self):
        self.gamma = 0.9
        self.eps = 0.1
        self.alpha = 0.1
        self.action_size = 4

        random_actions = {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
        self.pi = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = []

    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def add(self, state, action, reward):
        data = (state, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    def update(self):
        G = 0
        for data in reversed(self.memory):
            state, action, reward = data
            G = self.gamma * G + reward
            key = (state, action)
            self.Q[key] += (G - self.Q[key]) * self.alpha
            self.pi[state] = greedy_probs(self.Q, state, self.eps)

固定値 $\alpha$ で更新すると、各データに対する重みが指数関数的に増加する(指数移動平均)。  
これによって新しいデータほど重みが大きくなる。

### 修正後のモンテカルロ法を使った方策制御の実装(*コーディング*)

以下の2点の改善を行ったモンテカルロ法の方策制御を実装した。

- `greedy_probs()` 関数の最後を、完全な greedy ではなく ε-greedy にする
- `McAgent` クラスの `update()` メソッドの Q の更新は「固定値 $\alpha$ 方式」で行う

この `McAgent` クラスを使って実験をしてみる。

In [ ]:
# インスタンスの生成
env = GridWorld()
agent = McAgent()


episodes = 1000
for episode in range(episodes):
    state = env.reset()
    agent.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.add(state, action, reward)
        if done:
            agent.update()
            break

        state = next_state

print(agent.Q)

## 方策オフ型と重点サンプリング

モンテカルロ法に ε-greedy 法を組み合わせることで最適に近い方策を得ることが出来た。  
しかし、それは完全に最適方策ではない。  
ここでは、モンテカルロ法を使って完全に最適な方法を学習する方法について考える。  
その上で、方策オン型とオフ型について説明する。

### 方策オン型とオフ型

自分とは別の場所で得られた経験から自分の方策を改善するというアプローチを、強化学習では<font color="red">**方策オフ型**</font>(off-policy)という。  
一方、自分で得た経験から自分の方策を改善する場合は<font color="red">**方策オン型**</font>(on-policy)という。  
エージェントの方策は、役割的に見ると、2つの方策があると見做せる。  
1つは評価と改善の対象となる方策であり、<font color="red">**ターゲット方策**</font>(Target Policy)と呼ばれる。  
もう1つは、エージェントが実際に行動を起こす際に使う方策であり、<font color="red">**挙動方策**</font>(Behaviour Policy)と呼ばれる。  
我々はこれまで、「ターゲット方策」と「挙動方策」を区別せずに使ってきた。  
このようにターゲット方策と挙動方策を同じものと見做せる場合、それは方策オン型である。  
一方で、これら2つを分けて考えなければいけない場合は、方策オフ型である。  
この方策オフ型であれば、挙動方策には「探索」を行わせ、ターゲット方策には「活用」だけを行わせることができる。  
ただし、挙動方策から得られたサンプルデータを使ってターゲット方策に関連する期待値を求めるには計算に工夫が必要になる。  
ここで登場するのが<font color="red">**重点サンプリング**</font>(Importance Sampling)というテクニックである。

### 重点サンプリング(*コーディング*)

重点サンプリングは、ある確率分布の期待値を別の確率分布からサンプリングしたデータを使って計算する手法である。  
ここでは、重点サンプリングを説明するにあたって、単純な例として $\mathbb{E}_\pi[x]$ という期待値の計算について考える。  
$x$ は確率変数であり、 $x$ の確率は $\pi(s)$ で表すことにする。

$$\mathbb{E}_\pi[x] = \sum x\pi(x)$$

この期待値をモンテカルロ法で近似するには、 $x$ を確率分布 $\pi$ からサンプリングしてその平均を取る。

$$
\text{sampling: } x^{(i)} \sim \pi \quad (i = 1, 2, \cdots, n) \\
\mathbb{E}_\pi[x] \simeq \frac{x^{(1)} + x^{(2)} + \cdots + x^{(n)}}{n}
$$

ここで、 $x$ が別の確率分布 $b$ からサンプリングされた場合を考える。  
期待値 $\mathbb{E}_\pi[x]$ を次のように式変形する。

\begin{eqnarray*}
\mathbb{E}_\pi[x] &=& \sum x\pi(x) \\
&=& \sum x \frac{b(x)}{b(x)} \pi(x) \\
&=& \sum x \frac{\pi(x)}{b(x)} b(x) \\
&=& \mathbb{E}_b \left \lbrack x \frac{\pi(x)}{b(x)} \right \rbrack
\end{eqnarray*}

このようにすることで、確率分布 $b$ に関する期待値として表せるようになる。  
また、各 $x$ には $\frac{\pi(x)}{b(x)}$ が乗算されており、 $\rho(x) = \frac{\pi(x)}{b(x)}$ とすれば各 $x$ には「重み」として $\rho(x)$ がかけられていると見做せる。  
以上より、

$$
\text{sampling: } x^{(i)} \sim b \quad (i = 1, 2, \cdots, n) \\
\mathbb{E}_\pi[x] \simeq \frac{\rho(x^{(1)})x^{(1)} + \rho(x^{(2)})x^{(2)} + \cdots + \rho(x^{(n)})x^{(n)}}{n}
$$

これで $\pi$ とは異なる確率分布 $b$ からサンプリングしたデータを使って $\mathbb{E}_\pi[x]$ を計算することができる。  
重点サンプリングを実装してみる。  
初めに確率分布 $\pi$ の期待値を通常のモンテカルロ法で求める。

In [ ]:
# x と確率分布 pi の設定
x = np.array([1, 2, 3])
pi = np.array([0.1, 0.1, 0.8])

# 期待値
e = np.sum(x * pi)
print(f"E_pi[x]: {e}")

# モンテカルロ法
n = 100
samples = []
for _ in range(n):
    s = np.random.choice(x, p=pi)
    samples.append(s)

# 平均と分散の算出
mean = np.mean(samples)
var = np.var(samples)
print(f"MC: {round(mean, 3)}\t(var: {round(var, 3)})")

次に、重点サンプリングを使って期待値を求める。

In [ ]:
# 確率分布 b の設定
b = np.array([1/3, 1/3, 1/3])

# 重点サンプリング
n = 100
samples = []
for _ in range(n):
    idx = np.arange(len(b))
    i = np.random.choice(idx, p=b)
    s = x[i]
    rho = pi[i] / b[i]
    samples.append(rho * s)

# 平均と分散の算出
mean = np.mean(samples)
var = np.var(samples)
print(f"MC: {round(mean, 3)}\t(var: {round(var, 3)})")

平均は真値の2.7から少しだけ離れるものの、その近辺の値を取ることがわかる。  
一方で、分散についてはモンテカルロ法のときよりも大きいことがわかる。

### 分散を小さくする(*コーディング*)

分散が小さければ小さいほど少ないサンプル数で精度良く近似できる。  
逆に、分散が大きくなればなるほど、精度良く近似するにはサンプル数を増やす必要がある。  
分散を小さくする方法の1つが、2つの確率分布を近づけることである。

In [ ]:
# 変更後の確率分布 b の設定
b = np.array([0.2, 0.2, 0.6])

# 重点サンプリング
n = 100
samples = []
for _ in range(n):
    idx = np.arange(len(b))
    i = np.random.choice(idx, p=b)
    s = x[i]
    rho = pi[i] / b[i]
    samples.append(rho * s)

# 平均と分散の算出
mean = np.mean(samples)
var = np.var(samples)
print(f"MC: {round(mean, 3)}\t(var: {round(var, 3)})")

以上で分散が小さくなったことが確認できた。  
強化学習における主眼は、一方の方策に「探索」を、もう一方には「活用」を行わせることである。  
その条件を満たした上で、できるだけ2つの確率分布を近づけることで分散を小さくする。